In [3]:
import numpy as np
import gym
import gym_search

from tqdm import tqdm

NUM_ENVS = 4
NUM_STEPS = 1_000_000

rng = np.random.default_rng()
envs = gym.vector.make("Camera-v0", num_envs=NUM_ENVS, asynchronous=True)
pbar = tqdm()

envs.reset()

for _ in tqdm(range(0, NUM_STEPS, NUM_ENVS)):
    envs.step(rng.integers(0, envs.single_action_space.n, NUM_ENVS))
    pbar.update(NUM_ENVS)

4289it [00:14, 302.83it/s]
27664it [00:27, 1196.64it/s]/home/oslund/Thesis/repo/gym_search/envs/camera.py:137: RuntimeWarning: divide by zero encountered in true_divide
  p = np.array(position[:3]) / position[3]
38816it [00:37, 1119.71it/s]/home/oslund/Thesis/repo/gym_search/envs/camera.py:137: RuntimeWarning: divide by zero encountered in true_divide
  p = np.array(position[:3]) / position[3]
55036it [00:55, 1220.35it/s]/home/oslund/Thesis/repo/gym_search/envs/camera.py:137: RuntimeWarning: divide by zero encountered in true_divide
  p = np.array(position[:3]) / position[3]
  9%|▉         | 23499/250000 [01:39<15:56, 236.80it/s]


KeyboardInterrupt: 

93996it [01:51, 845.58it/s]